In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os

class RestaurantCrawler:
    def __init__(self):
        # 크롬 드라이버 설정
        self.chrome_options = Options()
        self.chrome_options.add_argument("--start-maximized")
        # self.chrome_options.add_argument("--headless")  # 필요시 헤드리스 모드 활성화
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                                       options=self.chrome_options)
        self.wait = WebDriverWait(self.driver, 3)
        
        csv_path = "../../data/external/google_gangnam_crawling_data.csv"

        # 결과 저장용 데이터프레임 - 기존 CSV 파일이 있으면 로드, 없으면 빈 데이터프레임 생성
        if os.path.exists(csv_path):
            self.restaurants_df = pd.read_csv(csv_path)
            self.csv_path = csv_path
        else:
            self.restaurants_df = pd.DataFrame(columns=[
                '음식점_이름','카테고리','음식점_사진','영업시간','전화번호',
                '리뷰','주소','음식점_태그','메뉴_정보'

            ]) # 빈 데이터프레임 생성
            self.csv_path = csv_path
    
    def search_google_maps(self, search_query):
        """구글 지도에서 음식점 검색 및 정보 수집"""
        try:
            self.driver.get("https://www.google.com/maps")
            
            # 검색창 찾기 및 검색어 입력
            search_box = self.wait.until(EC.presence_of_element_located(
                (By.ID, "searchboxinput")))
            search_box.clear()
            search_box.send_keys(search_query)
            search_box.send_keys(Keys.ENTER)
            
            # 검색 결과 대기
            time.sleep(2)

            # 현재 URL 확인하여 상세 페이지인지 목록 페이지인지 판단
            current_url = self.driver.current_url
            # 여러 결과 (목록 페이지인 경우) - 첫 번째 결과 클릭
            if "/place/" not in current_url:
                try:
                    # 첫 번째 결과 찾기
                    first_result = self.driver.find_element(By.CSS_SELECTOR, "a.hfpxzc")
                    
                    # 첫 번째 결과 클릭
                    first_result.click()
                    time.sleep(2)  # 상세 정보 로딩 대기
                except Exception as e:
                    print(f"검색 결과 클릭 중 오류: {e}")
            else:
                # 음식점 이름
                try:
                    restaurant_name = self.driver.find_element(
                        By.CSS_SELECTOR, "h1.DUwDvf").text
                except:
                    restaurant_name = search_query
                
                # 카테고리 정보
                try:
                    category = self.driver.find_element(
                        By.CSS_SELECTOR, "button.DkEaL ").text
                except:
                    category = ""
                
                # 음식점 사진 URL
                try:
                    photo_element = self.driver.find_element(
                        By.CSS_SELECTOR, "button[aria-label*='사진'] img")
                    photo_url = photo_element.get_attribute("src")
                except:
                    try:
                        photo_element = self.driver.find_element(
                            By.CSS_SELECTOR, ".RZ66Rb img[decoding='async']")
                        photo_url = photo_element.get_attribute("src")
                    except:
                        photo_url = ""

                # 영업시간 추출 (버튼 클릭 방식)
                try:
                    # 영업시간 버튼 찾기 및 클릭
                    hours_button = self.driver.find_element(By.CSS_SELECTOR, "div.OMl5r[aria-expanded='false']")
                    hours_button.click()
                    time.sleep(1)  # 테이블이 로드될 시간 부여
                    
                    # 영업시간 테이블 찾기
                    hours_table = self.driver.find_element(By.CSS_SELECTOR, "table.eK4R0e")
                    
                    # 요일별 영업시간 추출
                    business_hours = {}
                    rows = hours_table.find_elements(By.CSS_SELECTOR, "tr.y0skZc")
                    
                    for row in rows:
                        day = row.find_element(By.CSS_SELECTOR, "td.ylH6lf div").text
                        hours_list = row.find_elements(By.CSS_SELECTOR, "li.G8aQO")
                        hours_text = [hour.text for hour in hours_list]
                        business_hours[day] = hours_text
                    
                    # 딕셔너리 형태로 저장
                    hours = business_hours
                except Exception as e:
                    print(f"영업시간 추출 중 오류 발생: {e}")
                    hours = {}

                # 리뷰 (최대 5개)
                reviews = []
                try:
                    # 리뷰 섹션으로 이동
                    review_tab = self.driver.find_element(By.CSS_SELECTOR, "button[aria-label*='리뷰']")
                    review_tab.click()

                    time.sleep(1)
                    
                    # 스크롤 다운 함수 - 리뷰 더 로드
                    try:
                        # 리뷰 컨테이너 찾기 (여러 선택자 시도)
                        try:
                            scroll_container = self.driver.find_element(By.CSS_SELECTOR, "div.m6QErb-qJTHM-tJHJj")
                        except:
                            try:
                                scroll_container = self.driver.find_element(By.CSS_SELECTOR, "div[role='feed']")
                            except:
                                scroll_container = self.driver.find_element(By.XPATH, "//div[contains(@class, 'section-scrollbox')]")
                        
                        # 스크롤 다운 실행
                        for _ in range(3):  # 3번 스크롤 다운
                            self.driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll_container)
                            time.sleep(1.5)  # 로딩 대기 시간 증가
                    except Exception as e:
                        print(f"스크롤 다운 중 오류 발생: {e}")

                    
                    # 리뷰 요소들 수집
                    review_elements = self.driver.find_elements(
                        By.CSS_SELECTOR, "div.jftiEf")[:5]
                    
                    for review in review_elements:
                        try:
                            # 더보기 버튼이 있으면 클릭
                            more_button = review.find_element(By.CSS_SELECTOR, "button.w8nwRe")
                            more_button.click()
                            time.sleep(0.5)
                        except:
                            pass
                        
                        reviewer = review.find_element(By.CSS_SELECTOR, "div.d4r55").text
                        rating = review.find_element(
                            By.CSS_SELECTOR, "span.kvMYJc").get_attribute("aria-label")
                        content = review.find_element(By.CSS_SELECTOR, "span.wiI7pd").text
                        
                        reviews.append(f"{reviewer} - {rating}\n{content}")
                except Exception as e:
                    print(f"리뷰 수집 중 오류: {e}")
                
                return {
                    '음식점_이름': restaurant_name,
                    '카테고리': category,
                    '음식점_사진': photo_url,
                    '영업시간': hours,
                    '리뷰': "\n\n".join(reviews)
                }
            
        except Exception as e:
            print(f"구글 지도 크롤링 중 오류 발생: {e}")
            return {
                '음식점_이름': search_query,
                '카테고리': "",
                '음식점_사진': "",
                '영업시간': "",
                '리뷰': ""
            }
    
    def search_diningcode(self, restaurant_name):
        """다이닝코드에서 음식점 검색 및 정보 수집"""
        try:
            search_url = f"https://www.diningcode.com/list.dc?query={restaurant_name}"
            self.driver.get(search_url)
            
            # 검색 결과 대기
            time.sleep(1)
            
            # 첫 번째 검색 결과의 링크 가져오기
            try:
                first_result_link = self.wait.until(EC.presence_of_element_located(
                    (By.CSS_SELECTOR, ".Poi__List__Wrap a")))
                href = first_result_link.get_attribute('href')
                
                # 해당 URL로 직접 이동
                self.driver.get(href)
                time.sleep(2)
            except:
                print("다이닝코드 검색 결과가 없거나 링크를 찾을 수 없습니다.")
                return {'주소': "", '음식점_태그': "", '메뉴_정보': ""}
            
            # 주소 정보
            try:
                address = self.driver.find_element(By.CSS_SELECTOR, ".dc-spot-place").text
            except:
                address = ""
            
            # 전화번호 추출
            try:
                phone = self.driver.find_element(By.CSS_SELECTOR, ".dc-spot-tel").text
            except:
                phone = ""
            # 음식점 태그 정보
            try:
                tags = self.driver.find_element(By.CSS_SELECTOR, ".keywordStyle").text
            except:
                tags = ""
                
            # 메뉴 정보
            try:
                menu_info = self.driver.find_elements(By.CSS_SELECTOR, ".menu-info")
                menu_text = "\n".join([menu.text for menu in menu_info])
            except:
                menu_text = ""
            
            return {
                '주소': address,
                '전화번호': phone,
                '음식점_태그': tags,
                '메뉴_정보': menu_text
            }
            
        except Exception as e:
            print(f"다이닝코드 크롤링 중 오류 발생: {e}")
            return {
                '주소': "",
                '전화번호': "",
                '음식점_태그': "",
                '메뉴_정보': ""
            }
    
    def crawl_restaurant(self, search_query):
        """구글 지도와 다이닝코드에서 음식점 정보 수집"""
        # 구글 지도에서 정보 수집
        google_info = self.search_google_maps(search_query)
        
        # 다이닝코드에서 정보 수집 (구글에서 얻은 음식점 이름 사용)
        dining_info = self.search_diningcode(google_info['음식점_이름'])
        
        # 정보 합치기
        restaurant_info = {**google_info, **dining_info}
        
        # 데이터프레임에 추가
        self.restaurants_df = pd.concat([
            self.restaurants_df, 
            pd.DataFrame([restaurant_info])
        ], ignore_index=True)
        
        return restaurant_info
    
    def crawl_multiple_restaurants(self, search_queries):
        """여러 음식점 정보 수집"""
        for query in search_queries:
            print(f"'{query}' 정보 수집 중...")
            self.crawl_restaurant(query)
            time.sleep(2)  # 요청 간 딜레이
        
        return self.restaurants_df
    
    def save_to_csv(self, filename=None):
        """수집한 데이터를 CSV 파일로 저장"""
        save_path = filename or self.csv_path
        
        # 파일이 존재하고 데이터프레임이 비어있지 않은 경우
        if os.path.exists(save_path) and not self.restaurants_df.empty:
            # 헤더 없이 추가 모드로 저장 (기존 파일에 이어서 저장)
            self.restaurants_df.to_csv(save_path, mode='a', header=False, index=False, encoding='utf-8-sig')
        else:
            # 새 파일 생성 모드로 저장
            self.restaurants_df.to_csv(save_path, index=False, encoding='utf-8-sig')
        
        print(f"데이터가 {save_path}에 저장되었습니다.")
    
    def close(self):
        """드라이버 종료"""
        self.driver.quit()

# 사용 예시
if __name__ == "__main__":
    crawler = RestaurantCrawler()
    
    # 검색할 음식점 목록
    restaurants_to_search = [
        "써브웨이"
    ]
    
    try:
        # 여러 음식점 정보 수집
        results = crawler.crawl_multiple_restaurants(restaurants_to_search)
        
        # CSV 파일로 저장
        crawler.save_to_csv()
        
        print("크롤링 완료!")
        print(results)
    finally:
        # 드라이버 종료
        crawler.close()


'써브웨이' 정보 수집 중...
https://www.google.com/maps/search/%EC%8D%A8%EB%B8%8C%EC%9B%A8%EC%9D%B4?entry=ttu&g_ep=EgoyMDI1MDUwMy4wIKXMDSoASAFQAw%3D%3D
데이터가 ../../data/external/google_gangnam_crawling_data.csv에 저장되었습니다.
크롤링 완료!
     음식점_이름     카테고리                                             음식점_사진  \
0      써브웨이      NaN                                                NaN   
1      써브웨이      NaN                                                NaN   
2  서브웨이 학동점  샌드위치 가게  https://lh3.googleusercontent.com/gps-cs-s/AC9...   
3      써브웨이      NaN                                                NaN   
4      써브웨이      NaN                                                NaN   
5  서브웨이 학동점  샌드위치 가게  https://lh3.googleusercontent.com/gps-cs-s/AC9...   
6  서브웨이 학동점  샌드위치 가게  https://lh3.googleusercontent.com/p/AF1QipMuTT...   
7  서브웨이 학동점  샌드위치 가게  https://lh3.googleusercontent.com/p/AF1QipMuTT...   

                                                영업시간  \
0                                                 